In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [8]:
import os
import string


import numpy as np
import pandas as pd
from yelp_utils import load_sparse_csr
import yelp_utils
from __future__ import print_function

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import imdb

In [14]:
# SEED_VAL = 200
n_words=10000
# data_subset = "_10Percent"
VALIDATION_DATA_PERCENTAGE = 0.1
# WORK_DIR = os.getcwd()
# YELP_DATA_CSV_DIR = os.path.join(WORK_DIR, "data", "csv")
# YELP_DATA_WORD_2_VEC_MODEL_DIR = os.path.join(WORK_DIR, "data", "word2vec_model")
# YELP_DATA_SPARSE_MATRIX_DIR = os.path.join(WORK_DIR, "data", "sparse_matrix")

In [15]:
read_filename = os.path.join(yelp_utils.YELP_DATA_CSV_DIR, 'business_review_user'+ yelp_utils.data_subset+ '.csv')
df_data = pd.read_csv(read_filename, engine='c', encoding='utf-8')

In [16]:
def myLSTM(trainDataVecs, y,SEED_VAL=SEED_VAL):
    '''
    Function to train LSTM and print the accuracy for train and test.
    Based on code provided by Prof Mageed - https://github.com/mageed/deep_learning/blob/master/keras_tutorial_imdb.ipynb
    
    Divides the data in train 90% and test 10%. 
    
    Inputs
    trainDataVecs - Numpy darray matrix
    y - Numpy darray for vector
    SEED_VAL = seed for randomly shuffling the data
    
    Output
    prints the accuracy of trained model on training and testing data
    '''
    # Divide the data in test and train
    np.random.seed = SEED_VAL
    n_samples = len(trainDataVecs)
    sidx = np.random.permutation(n_samples)
    data_set_x = trainDataVecs.tolist()
    b = y
    
    # b[(b == 1) | (b == 2) | (b == 3)] = 0
    # b[(b == 4) | (b == 5)] = 1
    data_set_y = b.tolist()

    n_train = int(np.round(n_samples * (1. - VALIDATION_DATA_PERCENTAGE)))
    valid_set_x = [data_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [data_set_y[s] for s in sidx[n_train:]]
    train_set_x = [data_set_x[s] for s in sidx[:n_train]]
    train_set_y = [data_set_y[s] for s in sidx[:n_train]]

    # def remove_unk(x):
    #     return [[1 if w >= n_words else w for w in sen] for sen in x]

    # train_set_x = remove_unk(train_set_x)
    # valid_set_x = remove_unk(valid_set_x)

    train = (train_set_x, train_set_y)
    valid = (valid_set_x, valid_set_y)

    max_features = 100
    maxlen = trainDataVecs.shape[1]  # cut texts after this number of words (among top max_features most common words)
    batch_size = 32

    X_train, y_train=train[0], train[1]
    X_test, y_test= valid[0], valid[1]

    print("Pad sequences (samples x time)")
    # http://keras.io/preprocessing/sequence/
    X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
    X_test = sequence.pad_sequences(X_test, maxlen=maxlen, dtype='float32')

    y_train = np.array(y_train, dtype='int32')
    y_test = np.array(y_test, dtype='int32')

    print('Build model...')
    # http://keras.io/objectives/
    # http://keras.io/optimizers/

    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=maxlen))
    model.add(LSTM(128))  # try using a GRU instead, for fun
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # try using different optimizers and different optimizer configs
    # model.compile(loss='binary_crossentropy',
    #               optimizer='adam',
    #               class_mode="binary")

    # 'mean_squared_error', binary_crossentropy

    model.compile(loss='mean_squared_error', 
                  optimizer='adam', metrics=["accuracy"])

    print("Train...")
    %time model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=3, validation_data=(X_test, y_test))


    score1, accuracy1 = model.evaluate(X_train, y_train,
                                batch_size=batch_size,
                                show_accuracy=True)
    
    print('Train score:', score1)
    print ('Train Accuracy: ', accuracy1)
    
    score2, accuracy2 = model.evaluate(X_test, y_test,
                            batch_size=batch_size,
                            show_accuracy=True)
    
    print('Test score:', score2)
    print ('Test Accuracy: ', accuracy2)
    
    
y = np.array(df_data.review_stars.copy(), dtype='int32')

# Bag of words

In [17]:
spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR, "bagWords"+ data_subset)
bag_of_words_sparse_matrix = load_sparse_csr(spare_matrix_file + ".npz")

In [18]:
matrix_bag_of_words = bag_of_words_sparse_matrix.toarray()
myLSTM(matrix_bag_of_words, y, SEED_VAL)

Pad sequences (samples x time)
Build model...
Train...


DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpxgm_yx/da6ac6908198a9d83223e8a2be6b76b9.lib and object C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpxgm_yx/da6ac6908198a9d83223e8a2be6b76b9.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpa3xe3q/3822e09213127a9018d8f73e171845df.lib and object C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpa3xe3q/3822e09213127a9018d8f73e171845df.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_Gen

Train on 1974 samples, validate on 219 samples
Epoch 1/3
1974/1974 [==============================] - 131s - loss: 9.5641 - acc: 0.1145 - val_loss: 10.0141 - val_acc: 0.1050
Epoch 2/3
1974/1974 [==============================] - 131s - loss: 9.3202 - acc: 0.1170 - val_loss: 10.0139 - val_acc: 0.1050
Epoch 3/3
1974/1974 [==============================] - 130s - loss: 9.3200 - acc: 0.1170 - val_loss: 10.0138 - val_acc: 0.1050
Wall time: 20min 38s


C:\Anaconda\lib\site-packages\keras\models.py:426: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


1974/1974 [==============================] - 53s    
Train score: 9.31977784718
Train Accuracy:  0.117021276656
219/219 [==============================] - 5s     
Test score: 10.0138272521
Test Accuracy:  0.105022831254


# Bag of words + Hand craft features

In [19]:
# spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR, "bagWords_feat_add" + data_subset)
# feature_matrix_bag_of_words_and_hand_craft_features = load_sparse_csr(spare_matrix_file + ".npz")

In [20]:
# myLSTM(feature_matrix_bag_of_words_and_hand_craft_features.toarray(), y, SEED_VAL)

# Word Embedding

In [21]:
word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, "word2vec_feature_matrix" + data_subset+ ".csv")
feature_matrix_word2vec = np.genfromtxt(word2vec_feature_matrix_file, delimiter=',')   

In [22]:
myLSTM(feature_matrix_word2vec, y, SEED_VAL)

Pad sequences (samples x time)
Build model...
Train...


DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpwzdufs/46035747dd44bd1429349de5669a2bee.lib and object C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpwzdufs/46035747dd44bd1429349de5669a2bee.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpl1c3bj/79af18160b739f65153df56cfa3e90b7.lib and object C:/Users/hrushikesh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpl1c3bj/79af18160b739f65153df56cfa3e90b7.exp



Train on 1974 samples, validate on 219 samples
Epoch 1/3
1974/1974 [==============================] - 40s - loss: 9.7047 - acc: 0.1170 - val_loss: 8.9230 - val_acc: 0.0868
Epoch 2/3
1974/1974 [==============================] - 40s - loss: 9.4415 - acc: 0.1190 - val_loss: 8.9226 - val_acc: 0.0868
Epoch 3/3
1974/1974 [==============================] - 41s - loss: 9.4412 - acc: 0.1190 - val_loss: 8.9225 - val_acc: 0.0868
Wall time: 2min 22s
1974/1974 [==============================] - 16s    
Train score: 9.44090737495
Train Accuracy:  0.119047619123
219/219 [==============================] - 1s     
Test score: 8.92254887759
Test Accuracy:  0.0867579909016


# Word Embedding + Hand craft features

In [26]:
# word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, "word2vec_add_feature_matrix" + data_subset+ ".csv")
# feature_matrix_word2vec_and_hand_craft_features = np.genfromtxt(word2vec_feature_matrix_file, delimiter=',')  

In [27]:
# myLSTM(feature_matrix_word2vec_and_hand_craft_features, y, SEED_VAL)

# Hand craft features

In [28]:
# feature_matrix_hand_craft_features = feature_matrix_word2vec_and_hand_craft_features[:,100:104]

In [29]:
# myLSTM(feature_matrix_hand_craft_features, y, SEED_VAL)